In [14]:
import gffutils
import os
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool

In [15]:
gff_path = "input/gff/"
fasta_path = "input/fasta"
db_path = "intermediate/"
species_list = ["Bcur", "Bdor", "Bole", "Ccap"]

In [16]:
def get_db(species):
    gff_name = species + ".gff"
    if os.path.isfile(os.path.join(os.path.abspath(os.path.curdir),db_path + gff_name + ".db")):
        db = gffutils.FeatureDB(os.path.join(os.path.abspath(os.path.curdir),db_path + gff_name + ".db"))
    else:
        fn = gffutils.example_filename(os.path.join(os.path.abspath(os.path.curdir),gff_path + gff_name))
        db = gffutils.create_db(fn,
                                dbfn=db_path + gff_name + ".db",
                                force=True,
                                merge_strategy='merge',
                                id_spec=['ID', 'Name'])
    return (species, db)

In [17]:
with Pool(len(species_list)) as p: gff_dbs = {sp:db for sp,db in p.map(get_db, species_list)}

In [18]:
gff_dbs

{'Bcur': <gffutils.interface.FeatureDB at 0x104acb390>,
 'Bdor': <gffutils.interface.FeatureDB at 0x1064c8518>,
 'Bole': <gffutils.interface.FeatureDB at 0x106941e80>,
 'Ccap': <gffutils.interface.FeatureDB at 0x104acb2e8>}